# Concatenating COSA Datasets LOL

## Imports

In [1]:
import pandas as pd

In [ ]:
#initial dataset 
#air1 = pd.read_csv('med_center_air1.csv')

In [ ]:
#additional dataset
#air2 = pd.read_csv('med_center_air.csv')

In [ ]:
#check out datatypes and nulls
#air1.info()

In [ ]:
#checkout datatypes and nulls
#air2.info()

- O3 is a float in air 2, need to make them both floats

In [ ]:
#air1.astype({'O3': 'float64'}).dtypes

In [ ]:
#air1.astype({'O3': 'float64'})

In [ ]:
#air1.info()

In [ ]:
#air1.O3 = air1.O3.astype('float64')

In [ ]:
#air1.info()

In [ ]:
#air1.info()

## Full cosa dataset

In [2]:
import wrangle

In [3]:
def full_daily_COSA_dataframe():
    sound_df = wrangle.wrangle_sound()
    sound_df = sound_df.set_index('DateTime')
    sound_df = sound_df.sort_index()
    flood_df = wrangle.clean_flood()
    flood_df = flood_df.set_index('datetime')
    weather_df = wrangle.wrangle_weather()
    air_df = wrangle.clean_air()
    air_df.datetime = pd.to_datetime(air_df.datetime)
    air_df = air_df.set_index('datetime')
    air_df = air_df.sort_index()
    weather_day_df = weather_df.resample('D', on='datetime').mean()
    flood_day_df = flood_df.resample('D').mean()
    sound_day_df = sound_df.resample('D').mean()
    air_day_df = air_df.resample('D').mean().drop(columns = ['hour', 'weekday', 'CO_24hr', 'Pm_25_24hr', 'Pm_10_24hr', 'SO2', 'O3', 'NO2'])
    air2_5 = air_df.drop(air_df.columns.difference(['Pm2_5', 'AQI_pm2_5']), 1)
    air10 = air_df.drop(air_df.columns.difference(['Pm10', 'AQI_pm10']), 1)
    airCO = air_df.drop(air_df.columns.difference(['CO', 'AQI_CO']), 1)
    series2_5 = air2_5.resample('D').max().rename(columns = {'AQI_pm2_5': 'most_hazardous_pm2.5_level'})['most_hazardous_pm2.5_level']
    series10 = air10.resample('D').max().rename(columns = {'AQI_pm10': 'most_hazardous_pm10_level'})['most_hazardous_pm10_level']
    seriesCO = airCO.resample('D').max().rename(columns = {'AQI_CO': 'most_hazardous_CO_level'})['most_hazardous_CO_level']
    hazards = pd.DataFrame(series2_5).join(series10).join(seriesCO)
    df = weather_day_df.join(air_day_df).join(hazards).join(sound_day_df).join(flood_day_df)
    df = df.round({'celsius': 2, 'farenheit': 2, 'humidity': 2, 'dewpoint_celsius': 2, 'dewpoint_farenheit': 2,
          'pressure': 2, 'NoiseLevel_db': 2, 'sensor_to_water_feet': 2, 'sensor_to_water_meters': 2,
          'sensor_to_ground_feet': 2, 'sensor_to_ground_meters': 2, 'flood_depth_feet': 2,
          'flood_depth_meters': 2})
    return df

In [4]:
df = full_daily_COSA_dataframe()


In [5]:
df.head()

,celsius,farenheit,humidity,dewpoint_celsius,dewpoint_farenheit,pressure,Pm1_0,Pm2_5,Pm10,CO,most_hazardous_pm2.5_level,most_hazardous_pm10_level,most_hazardous_CO_level,NoiseLevel_db,sensor_to_water_feet,sensor_to_water_meters,sensor_to_ground_feet,sensor_to_ground_meters,flood_depth_feet,flood_depth_meters
datetime,,,,,,,,,,,,,,,,,,,,
2021-04-20,20.89,69.56,45.79,7.67,45.80,988.08,5.193732,8.022792,7.324786,0.299145,Hazardous,Good,Good,66.23,13.24,4.04,13.5,4.12,0.26,0.08
2021-04-21,15.32,59.60,36.49,-0.31,31.43,985.50,2.968750,3.616477,4.460227,0.144886,Unhealthy,Moderate,Good,66.66,13.34,4.07,13.5,4.12,0.16,0.05
2021-04-22,16.00,60.85,67.74,9.64,49.35,979.65,16.866477,18.247159,22.963068,0.056818,Unhealthy for Sensitive Groups,Moderate,Good,66.20,13.30,4.05,13.5,4.12,0.21,0.06
2021-04-23,22.51,72.46,85.76,19.65,67.37,971.28,18.123563,19.209770,23.916667,0.166667,Unhealthy for Sensitive Groups,Moderate,Good,66.40,13.00,3.96,13.5,4.12,0.50,0.15
2021-04-24,24.97,76.97,50.29,10.99,51.78,974.48,3.804533,4.458924,51.232295,0.405099,Moderate,Good,Hazardous,64.81,13.13,4.00,13.5,4.12,0.37,0.11


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 80 entries, 2021-04-20 to 2021-07-08
Freq: D
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   celsius                     80 non-null     float64 
 1   farenheit                   80 non-null     float64 
 2   humidity                    80 non-null     float64 
 3   dewpoint_celsius            80 non-null     float64 
 4   dewpoint_farenheit          80 non-null     float64 
 5   pressure                    80 non-null     float64 
 6   Pm1_0                       80 non-null     float64 
 7   Pm2_5                       80 non-null     float64 
 8   Pm10                        80 non-null     float64 
 9   CO                          80 non-null     float64 
 10  most_hazardous_pm2.5_level  80 non-null     category
 11  most_hazardous_pm10_level   80 non-null     category
 12  most_hazardous_CO_level     80 non-null     category